- Họ và tên: Phạm Trần Yến Quyên
- MSSV: 22127357
- Lớp: 22CLC02

# PROJECT 4 - Chéo hóa ma trận.
- Input: Ma trận $A$.
- Output: Ma trận chéo hóa từ ma trận $A$.

## TEST CASES

In [1]:
bt1 = [
    [-1,3],
    [-2,4]
]

bt2 = [
    [5,2],
    [9,2]
]

bt3 = [
    [1, -1, -1],
    [1, 3, 1],
    [-3, 1, -1]
]

bt4 = [
    [5, -1, 1],
    [-1, 2, -2],
    [1, -2, 2]
]

bt5 = [
    [1,3,3],
    [-3,-5,-3],
    [3,3,1]
]

bt6 = [
    [4,0,-1],
    [0,3,0],
    [1,0,2]
]

bt7 = [
    [3,4,-4],
    [-2,-1,2],
    [-2,0,1]
]

bt8 = [
    [0,0,-2],
    [1,2,1],
    [1,0,3]
]

bt9 = [
    [1,0,0],
    [1,2,0],
    [-3,5,2]
]

bt10 = [
    [4,0,1],
    [-2,1,0],
    [-2,0,1]
]

bt = [bt1, bt2, bt3, bt4, bt5, bt6, bt7, bt8, bt9, bt10]

`Diagonalize()`

In [2]:
from fractions import Fraction
from sympy import Matrix, Symbol, solve
import numpy as np

class matrix:
    def __init__(self, matrix):
        self.matrix = matrix
        self.m = len(matrix) # Số hàng
        self.n = len(matrix[0]) # Số cột
        
    def if_square(self):
        # Kiểm tra ma trận vuông
        if len(self.matrix) == len(self.matrix[0]):
            return 1
        return 0

    def CharacteristicPolynomial(self):
        # Tính định thức của ma trận A - λI = 0
        eq = np.polynomial.Polynomial.fromroots(np.linalg.eigvals(self.matrix))  # .fromroots trả về đa thức với các nghiệm là các giá trị riêng của ma trận
        for i in range(len(eq.coef)):
            eq.coef[i] = Fraction(eq.coef[i]).limit_denominator() # Vì lỗi sai số ở BT5 nên chuyển về dạng phân số
        return eq
    
    def calEigenValue(self):
        # Tìm các giá trị riêng của ma trận A
        eq = self.CharacteristicPolynomial()
        x = Symbol('x')
        # Tìm nghiệm của đa thức bằng sympy
        ans = solve(eq(x), x) 
        return ans
    
    def calDiagTothePower(self, n):
        # Tính ma trận A^n
        return [[self.matrix[i][j]**n for i in range(self.m)] for j in range(self.n)]
    
    def calEigenVector(self):
    # Tìm các vector riêng từ trị riêng của ma trận A
        eigenvalues = self.calEigenValue()
        eigenvectors = []
        
        # Chuyển ma trận về dạng sympy để tính nullspace
        A = Matrix(self.matrix)
        for eigenvalue in eigenvalues:
            # Thay các giá trị riêng vào ma trận A - λI
            A_minus_lambda_I = A - eigenvalue * Matrix.eye(self.m)
            # Tìm không gian con của ma trận A - λI
            null_space = A_minus_lambda_I.nullspace()
            if null_space:
                for vec in null_space:
                    # Chuyển vector riêng về dạng float (tranh lỗi sai số)
                    eigenvectors.append(np.array(vec.evalf()).astype(float))
        
        # Loại bỏ các vector trùng nhau (vì nullspace trả về các vector riêng không duy nhất)
        unique_eigenvectors = []
        for v in eigenvectors:
            if not any(np.allclose(v, u) for u in unique_eigenvectors): # Kiểm tra xem vector đã tồn tại trong list chưa
                unique_eigenvectors.append(v)
        return unique_eigenvectors
    
    def Diagonalize(self):
        # Tìm ma trận đường chéo D bằng cách xây P từ các vector riêng và tính P^-1AP                
        # Phải là ma trận vuông mới chéo hóa được
        if not self.if_square():
            return "Không thể chéo hóa vì ma trận không vuông"
        
        eigenvectors = self.calEigenVector()
        # Nếu số vector riêng không đủ để chéo hóa
        if len(eigenvectors) < self.m:
            return "Không the chéo hóa vì không đủ vector riêng"
        
        # Lập ma trận P từ các vector riêng (cột của P là các vector riêng)
        P = np.column_stack(eigenvectors) # P = [v1, v2, ..., vn]
        
        # Kiểm tra xem ma trận P có khả nghịch không
        if np.linalg.matrix_rank(P) < self.m:
            return "Không thể chéo hóa vì ma trận P không khả nghịch"
        
        # Nghịch đảo ma trận P  
        P_inv = np.linalg.inv(P)
        
        # Tính D = P^-1AP
        D = np.dot(np.dot(P_inv, self.matrix), P)
        
        # Chữa lỗi sai số ở D (tương tự như ở hàm CharacteristicPolynomial())
        for i in range(len(D)):
            for j in range(len(D[0])):
                D[i][j] = Fraction(D[i][j]).limit_denominator()
        return D

        """ eigenvalues = self.calEigenValue()
        D =[[0 for i in range(self.n)] for j in range(self.m)]
        if (len(eigenvalues) != self.m):
            return "Ma trận không chéo hóa được do số lượng giá trị riêng không bằng số hàng của ma trận"
        for i in range(self.m):
            D[i][i] = eigenvalues[i] """
        
        
def print_matrix(matrix):
    # In lại ma trận để thoát format của sympy
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            # Kiểm tra có phải là chữ không (ko chéo hóa được)
            if type(matrix[i][j]) == str:
                for k in range(len(matrix)):
                    for l in range(len(matrix[0])):
                        print(matrix[k][l], end = '')
                return
            if j == 0:
                print("[", end = '')
            print(matrix[i][j], end = '')
            if j == len(matrix[0]) - 1:
                print("]")
            else:
                print(", ", end = '')
            


## Ứng dụng của chéo hóa ma trận trong tính toán  Markov Chain/Markov Process (Xích Markov) 
### Định nghĩa
- **Markov Chain**: Markov Process là cơ sở cho phương pháp mô phỏng ngẫu nhiên Markov Chain Monte Carlo (MCMC - Metropolis–Hastings Algorithm), được dùng để mô phỏng việc lấy mẫu từ một phân bố xác suất phức tạp.
- **Ứng dụng**: Markov Process rất hữu ích trong nhiều lĩnh vực khác nhau, đặc biệt là trong việc dự đoán và phân tích sự chuyển dịch dân số, thị trường, và các hệ thống động khác. Một số ứng dụng cụ thể bao gồm:
1. **Kinh tế và tài chính**: Dự đoán sự thay đổi trong các thị trường tài chính, như dự đoán giá cổ phiếu hoặc tỉ giá hối đoái.
2. **Quản lý dân số và đô thị**: Dự đoán sự thay đổi dân số trong các khu vực đô thị và nông thôn để quy hoạch hạ tầng, y tế, giáo dục.
3. **Mô phỏng hệ thống sinh thái**: Dự đoán sự thay đổi trong các quần thể sinh vật và tác động của các yếu tố môi trường.
4. **Lý thuyết thông tin và trí tuệ nhân tạo**: Dự đoán các trạng thái tiếp theo trong các hệ thống học máy và các thuật toán AI.

### Bài toán dự đoán dân số các miền trong tương lai
- Giả sử ta có một bến xe đi 3 miền Nam - Trung - Bắc, hằng năm sẽ có số lượng người di chuyển giữa 3 miền để vào cư trú/sinh sống.
- Để đơn giản hóa mô hình bài toán, ta sẽ bỏ qua các yếu tố ngoại ảnh hưởng đến số lượng người di chuyển trong thời gian($t$ - năm) như:
    + Những người vào/ra khỏi vùng có nguồn gốc từ các nơi khác không phải 3 miền trên.
    + Số ca sinh/tử. 
- vậy sẽ được giả định là tổng dân số là **không đổi** và mọi người di chuyển giữa 3 miền Nam - Trung - Bắc. Thiết lập/thu thập bảng dữ liệu giả định, ta ước tính được cách dân số thay đổi ở các miền từ năm này sang năm khác dưới dạng ***transition matrix $3\times 3$*** (Ma trận chuyển đổi) như sau:
\begin{bmatrix}
từ Nam      & từ Trung     & từ Bắc \\
\end{bmatrix}

\begin{equation}
    A = 
    \begin{bmatrix}
        0.75     & 0.10    &   0.10    & tới Nam\\
        0.10     & 0.60    &   0.20    & tới Trung\\
        0.15     & 0.30    &   0.70    & tới Bắc\\
    \end{bmatrix}
\end{equation}

- **Trong đó**:
    + Giá trị từng cột từ trái sang phải cho ta biết phần trăm (%) người dân ở khu vực Nam/Trung/Bắc di chuyển tới sinh sống ở Nam/Trung/Bắc. 
    + Tổng của mỗi cột là 1 tương ứng với 100% số dân sinh sống tại khu vực ở tính ở cuối năm.
$$Sum(col1) = 1 | Sum(col2) = 1 | Sum(col3) = 1$$ 

- Giả sử số người đi các chuyến ra Nam/Trung/Bắc được biểu diễn dưới dạng vector cột là:
\begin{equation}
    S = 
    \begin{bmatrix}
        400000  & tới Nam\\
        1400000 & tới Trung\\
        200000  & tới  Bắc\\
    \end{bmatrix}
\end{equation}

- Để tính toán dân số năm kế tiếp, ta nhân ma trận $A$ với vector $S$:
> $$S_{new} = A \times S$$

- Thực hiện phép nhân ma trận này, ta có:
\begin{equation}
    S_{new} = 
    \begin{bmatrix}
    0.75 & 0.10 & 0.10 \\
    0.10 & 0.60 & 0.20 \\
    0.15 & 0.30 & 0.70 \\
    \end{bmatrix} 
    \times
    \begin{bmatrix}
    400,000 \\
    1,400,000 \\
    200,000 \\
    \end{bmatrix}
\end{equation}

Tính toán từng phần tử của vector $S_{new}$:
\begin{aligned}
S_{new,1} & = (0.75 \times 400,000) + (0.10 \times 1,400,000) + (0.10 \times 200,000) = 300,000 + 140,000 + 20,000 = 460,000 \\
S_{new,2} & = (0.10 \times 400,000) + (0.60 \times 1,400,000) + (0.20 \times 200,000) = 40,000 + 840,000 + 40,000 = 920,000 \\
S_{new,3} & = (0.15 \times 400,000) + (0.30 \times 1,400,000) + (0.70 \times 200,000) = 60,000 + 420,000 + 140,000 = 620,000 \\
\end{aligned}

Vậy dân số dự đoán của ba miền trong năm kế tiếp là:
\begin{equation}
    S_{new} = 
    \begin{bmatrix}
    460,000 & \text{tới Nam} \\
    920,000 & \text{tới Trung} \\
    620,000 & \text{tới Bắc} \\
    \end{bmatrix}
\end{equation}

**Tính toán dân số sau nhiều năm**
- Nếu ta muốn tính toán dân số sau $t$ năm, ta sẽ thực hiện phép nhân ma trận $A$ với vector dân số $S$ ban đầu $t$ lần, hoặc tương đương với việc nhân $A$ với $S$ và sau đó nhân với kết quả, lặp lại $t$ lần. Điều này có thể được biểu diễn dưới dạng: 
> $$S_t = A^t * S$$
- Trong đó $A^t$ là ma trận chuyển đổi $A$ sau $t$ lần nhân ma trận.

### Áp dụng chéo hóa vào bài toán
- Để tính toán hiệu quả hơn và có thể khai thác các tính chất đặc biệt của ma trận, ta có thể sử dụng phương pháp chéo hóa ma trận. Một ma trận vuông $A$ có thể được chéo hóa nếu tồn tại một ma trận khả nghịch $P$ sao cho:
$$A = P D P^{-1}$$

- Trong đó $D$ là một ma trận đường chéo chứa các giá trị riêng của $A$. Khi đó, để tính toán $A^t$, ta có thể sử dụng:
$$A^t = (P D P^{-1})^t = P D^t P^{-1}$$

- Do $D$ là ma trận đường chéo, việc tính $D^t$ trở nên rất đơn giản: chỉ cần lấy lũy thừa $t$ của từng phần tử trên đường chéo.

Giả sử $A$ có các giá trị riêng $\lambda_1, \lambda_2, \lambda_3$, và các vector riêng tương ứng là $v_1, v_2, v_3$, ta có thể biểu diễn $D$ và $P$ như sau:
\begin{equation}
D = 
\begin{bmatrix}
    \lambda_1 & 0 & 0 \\
    0 & \lambda_2 & 0 \\
    0 & 0 & \lambda_3 \\
\end{bmatrix}
\end{equation}

\begin{equation}
P = 
\begin{bmatrix}
    v_1 & v_2 & v_3 \\
\end{bmatrix}
\end{equation}

- Với việc chéo hóa ma trận, ta có thể tính toán $A^t$ một cách hiệu quả và từ đó dự đoán dân số sau $t$ năm:
> $$S_t = A^t S = P D^t P^{-1} S$$

→ Quá trình này giúp tối ưu hóa các phép tính ma trận và đưa ra kết quả chính xác hơn trong các bài toán dự đoán dựa trên mô hình Markov.

### Kết luận:
> Ứng dụng của chéo hóa trong Markov Chain/Markov Process giúp ta phân tích và dự đoán được sự biến đổi của các mô hình Markov Chain qua nhiều giai đoạn thời gian một cách dễ dàng hơn. Khi chéo hóa một ***transition matrix***, ta có thể tìm ra các giá trị riêng và vector riêng tương ứng. Điều này giúp ta hiểu rõ hơn về trạng thái ổn định của mô hình (steady state) và tốc độ đi đến đến trạng thái này. Ngoài ra, chéo hóa ma trận còn giúp ta đơn giản hóa các phép tính lũy thừa của ma trận, từ đó dễ dàng dự đoán các trạng thái trong tương lai.

### Nguồn tham khảo:
- https://www.math.wustl.edu/~freiwald/309markov.pdf
- https://www.youtube.com/watch?v=oKhAS7Xret0

In [3]:
import numpy as np

def MarkovChainExample(t):
    # Giải bài toán Bến xe buýt bằng cách ứng dụng chéo hóa ma trận
    # Transition matrix A
    trans_matrix = [
        [75, 10, 10],
        [10, 60, 20],
        [15, 30, 70]
    ] 
    # Các số ở dạng phần trăm (vì hàm chéo hóa không xử lý số thực)
    # Nhân mỗi phần tử cho 100 để chuyển về số nguyên
    # Phía dưới sẽ chia cho 100 để chuyển về dạng phần trăm

    # Số lượng hành khách tại các bến xe tại thời điểm ban đầu
    S = [400000, 1400000, 200000]    
    
    A = matrix(trans_matrix)
    # Chéo hóa ma trận A
    D = A.Diagonalize()

    # Kiểm tra xem có chéo hóa được không, được thì chia cho 100 để chuyển về dạng phần trăm
    if type(D) == str:
        print(D)
    else:
        # Chuyển ma trận D về dạng phần trăm
        D = [[D[i][j] / 100 for j in range(len(D[0]))] for i in range(len(D))]
    
    # Tính ma trận P từ các vector riêng
    P = np.column_stack(A.calEigenVector())
    P = [[P[i][j] / 100 for j in range(len(P[0]))] for i in range(len(P))]
    
    # Tính ma trận nghịch đảo của P
    P_inv = np.linalg.inv(P)
    
    # Tính A^t bằng cách tính D^t (với D là ma trận đường chéo) và nhân với P và P^-1
    # A^t = P * D^t * P^-1
    A_t = P @ np.linalg.matrix_power(np.diag(np.diag(D)), t) @ P_inv
    # Tính số lượng hành khách tại các bến xe sau t thời điểm
    S_t = A_t @ S

    return S_t

# MAIN 
## Input: Ma trận vuông $A$
## Output: Ma trận đã chéo hóa (nếu có thể) & kết quả ví dụ ứng dụng.

In [4]:
def main():    
    # Bài tập chéo hóa
    for i in range(len(bt)):
        print("Bt", i+1, ":")
        A = matrix(bt[i])
        """ print("Phương trình đặc trưng:", A.CharacteristicPolynomial())        
        print(A.calEigenValue())  
        print(A.calEigenVector()) """
        print("Ma trận D:")
        print_matrix(A.Diagonalize())
        print("\n=======================================")
        
    # Ứng dụng chéo hóa
    t = 10
    print(f"Dân số ở mỗi miền sau thời gian {t} năm là:")
    print(MarkovChainExample(t))

if __name__ == "__main__":
    main()


Bt 1 :
Ma trận D:
[1.0, 0.0]
[0.0, 2.0]

Bt 2 :
Ma trận D:
[-1.0, 0.0]
[0.0, 8.0]

Bt 3 :
Ma trận D:
[-2.0, 0.0, 0.0]
[0.0, 2.0, 0.0]
[0.0, 0.0, 3.0]

Bt 4 :
Ma trận D:
[0.0, 0.0, 0.0]
[0.0, 3.0, 0.0]
[0.0, 0.0, 6.0]

Bt 5 :
Ma trận D:
[-2.0, 0.0, 0.0]
[0.0, -2.0, 0.0]
[0.0, 0.0, 1.0]

Bt 6 :
Ma trận D:
Không the chéo hóa vì không đủ vector riêng
Bt 7 :
Ma trận D:
[-1.0, 0.0, 0.0]
[0.0, 1.0, 0.0]
[0.0, 0.0, 3.0]

Bt 8 :
Ma trận D:
[1.0, 0.0, 0.0]
[0.0, 2.0, 0.0]
[0.0, 0.0, 2.0]

Bt 9 :
Ma trận D:
Không the chéo hóa vì không đủ vector riêng
Bt 10 :
Ma trận D:
[1.0, 0.0, 0.0]
[0.0, 2.0, 0.0]
[0.0, 0.0, 3.0]

Dân số ở mỗi miền sau thời gian 10 năm là:
[569120.67256949 572431.4227482  858447.9046823 ]


# Nhận xét chung:

## Thuật toán chéo hóa ma trận:
### Ý tưởng chính:
1. Lập ma trận $|A - D I|$: Các thành phần trên đường chéo sẽ trừ thêm cho λ.
\begin{bmatrix}
    n_{11}-λ      & n_{12}        & \cdots  &  n_{1n}   \\
    n_{21}        & n_{22}-λ      & \cdots  &  n_{2n}   \\
    \vdots        & \vdots        & \vdots  &  \vdots   \\
    n_{m1}        &  n_{m2}       & \cdots  &  n_{mn}-λ \\
\end{bmatrix}

2. Tìm các vector riêng tương ứng với trị riêng $D$ tìm được:
    - Bước 1: Giải nghiệm $|A - D I| = 0$ thu được trị riêng: {$λ_1$, $λ_2$,...$λ_n$}
    - Bước 2: Bước 3: Tìm $null(A - \lambda_i I_n)$. Thay và giải các hệ phương trình $|A - λ_i I| = 0$ tìm vector riêng. 
    **VD**: Với $\lambda_i = 3$, giải $|A - 3 I| = 0$
> Chỉ khi có đủ $n$ vector riêng độc lập tuyến tính thì ma trận chéo hóa được, ngược lại thông báo không chéo hóa được.


3. Lập ma trận trực giao $P$ có các vector riêng là cột:
\begin{equation}
    P = {v1, v2,..., A_n} = 
    \begin{bmatrix}
        A_{11}    & A_{21}    & \cdots  &  A_{n1}   \\
        A_{12}    & A_{22}    & \cdots  &  A_{n2}   \\
        \vdots    & \vdots    & \vdots  &  \vdots   \\
        A_{1m}    & A_{2m}    & \cdots  &  A_{nm}   \\
    \end{bmatrix}
\end{equation}

4. Tìm ma trận chéo qua công thức 
> $$P^{-1} A P= D$$

### Các cách tìm định thức ma trận:
- **N = 1:**
\begin{bmatrix}
    A_{11} \\
\end{bmatrix}
> $$|A| = A_{11}$$
- **N = 2:** 
\begin{bmatrix}
    A_{11}      & A_{12}    \\
    A_{21}      & A_{22}    \\
\end{bmatrix}
> $$|A| = A_{11}A_{22} - A_{12}A_{21}$$


- **N = 3:**
\begin{bmatrix}
    A_{11}      & A_{12}  & A_{13} \\
    A_{21}      & A_{22}  & A_{23} \\
    A_{31}      & A_{32}  & A_{33} \\
\end{bmatrix}

    + Cách 1: Sử dụng Saurrus
        - Bước 1: Lập ma trận theo phương pháp Sarrus.
\begin{bmatrix}
    A_{11} & A_{12}  & A_{13} & | &  A_{11} & A_{12} \\
    A_{21} & A_{22}  & A_{23} & | &  A_{21} & A_{22} \\
    A_{31} & A_{32}  & A_{33} & | &  A_{31} & A_{32} \\
\end{bmatrix}

        - Bước 2: Giải phương trình có được từ tổng tích của các đường chéo xuống vào nhau rồi trừ đi tổng các tích của các đường chéo lên.
> $$|A| = [(A_{11}A_{22}A_{33}) + (A_{12}A_{23}A_{31}) + (A_{13}A_{21}A_{32})] - [(A_{12}A_{21}A_{33}) + (A_{11}A_{23}A_{32}) + (A_{13}A_{22}A_{31})]$$


- **N >= 3:**
\begin{bmatrix}
    A_{11}    & \cdots  &  A_{1n}   \\
    A_{21}    & \cdots  &  A_{2n}   \\
    \vdots    & \vdots  &  \vdots   \\
    A_{n1}    & \cdots  &  A_{nn}   \\
\end{bmatrix}
    + Cách 1: Sử dụng ma trận phụ hợp.
    + Cách 2: Sử dụng công thức khai triển, chọn hàng $i$ bất kỳ (nên chọn nhiều phần tử là 0 nếu có). 
> $$|A| = \sum_{j=1}^n A_{ij} (-1)^{i+j} |A_{i|j}|$$

- **Ma trận bậc thang:**
\begin{bmatrix}
    A_{11}    & \cdots  &  0   \\
    0         & A_{22}  &  0   \\
    \vdots    & \vdots  &  \vdots   \\
    0         & \cdots  &  A_{nn}   \\
\end{bmatrix}
> $$|A| = A_{11}A_{22}...A_{nn}$$